In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/weather-stations/kelowna/kelowna_stations.csv')

In [3]:
df.sort_values(by=['Latitude'], ascending = True)

,Network ID,Network Name,Native ID,Station ID,Station Name,History ID,Province,Longitude,Latitude,Elevation (m),Record Start,Record End,Obs Freq,Variables
78,1,EC,1131410,1112,CARMI,1515,BC,-119.083333,49.500000,1245.0,1924-01-01,1969-03-31,Daily,Precipitation Amount|Rainfall Amount|Snowfall ...
7,1,EC,1126160,243,PENTICTON SEWAGE PLANT,646,BC,-119.600000,49.500000,344.0,1954-10-01,1969-11-30,Daily,Precipitation Amount|Rainfall Amount|Snowfall ...
285,17,ARDA,112254,4998,ROGER RCH DS,6678,BC,-119.777500,49.513333,832.0,1980-10-08,1981-07-22,Daily,Precipitation Amount|Surface Snow Depth (Point)
145,12,FLNRO-WMB,328,1876,PENTICTON RS,2279,BC,-119.553300,49.518300,427.0,1988-08-16,2023-10-08,Hourly,Dew Point Temperature|Precipitation Amount|Pre...
165,12,FLNRO-WMB,393,1936,NICOLL,2339,BC,-118.360300,49.526700,866.0,1988-10-28,2024-01-14,Hourly,Dew Point Temperature|Precipitation Amount|Pre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,1,EC,1164830,1287,MALAKWA EAGLE R HATCHERY,1690,BC,-118.650000,51.000000,404.0,1924-01-01,2001-01-01,Daily,Precipitation Amount|Precipitation Climatology...
369,17,ARDA,116181,5660,UP ROSS CK KAM,7342,BC,-119.223611,51.000000,858.0,1971-05-12,1973-10-23,Daily,Precipitation (Cumulative)|Surface Snow Depth ...
131,1,EC,1176750,1362,REVELSTOKE,1765,BC,-118.200000,51.000000,456.0,1898-05-01,1969-11-30,Daily,Precipitation Amount|Rainfall Amount|Snowfall ...
167,12,FLNRO-WMB,696,2147,ZZ FOWLER LAKE,2550,BC,-120.000000,51.000000,13.0,1989-05-16,1990-10-05,Daily,Precipitation Amount|Relative Humidity (Mean)|...


In [4]:
longitude_max = df['Longitude'].max()
longitude_min = df['Longitude'].min()
latitude_max = df['Latitude'].max()
latitude_min = df['Latitude'].min()

In [5]:
print(longitude_max, longitude_min, latitude_max, latitude_min)

-118.193333333333 -120.330555555556 51.0 49.5


In [6]:
longitude_distance = longitude_max - longitude_min
latitude_distance = latitude_max - latitude_min
print(longitude_distance, latitude_distance)

2.137222222223002 1.5


In [7]:
burn_data = pd.read_csv("./data/satellite-burn/satellite_burn.csv")
burn_data

,YEAR,FIRE_SIZE_HA,LATITUDE,LONGITUDE
0,2013.0,0.009,54.572151,-124.521500
1,2013.0,2.400,54.59695,-124.596901
2,2013.0,2.500,54.597515,-125.080101
3,2013.0,6.100,54.567017,-124.495781
4,2013.0,0.200,54.655117,-124.710854
...,...,...,...,...
27516,2022.0,0.009,51.8735,-118.622600
27517,2022.0,0.200,54.098283,-122.937800
27518,2022.0,0.001,49.886617,-115.855800
27519,2022.0,0.009,49.644467,-120.525450


In [8]:
year_mask = (burn_data['YEAR'] == 2013)
burn_data = burn_data[year_mask]
total_burn = []

burn_data = burn_data.astype({'LATITUDE':'float', 'LONGITUDE': 'float'})

In [9]:
def is_between(target, my_min, my_max):
    return target >= my_min and target < my_max

In [10]:
def spatially_split(df, num_long_splits, num_lat_splits):
    longitude_max = df['Longitude'].max()
    longitude_min = df['Longitude'].min()
    latitude_max = df['Latitude'].max()
    latitude_min = df['Latitude'].min()
    longitude_distance = longitude_max - longitude_min
    latitude_distance = latitude_max - latitude_min
    
    long_split_distance = longitude_distance / num_long_splits
    lat_split_distance = latitude_distance / num_lat_splits
    
    long_track = longitude_min
    lat_track = latitude_min
    print(f"min_long = {longitude_min}, min_lat = {latitude_min}, max_long = {longitude_max}, max_lat = {latitude_max}, long_split = {long_split_distance}, lat_split = {lat_split_distance}")
    zero_count = 0
    num_above = 0
    
    for i in range(num_lat_splits):
        lat_track = latitude_min + i * lat_split_distance
        for j in range(num_long_splits):
            long_track = longitude_min + j * long_split_distance
            
            latitude_mask_station = (df['Latitude'] < lat_track + lat_split_distance) & (df['Latitude'] >= lat_track)
            longitude_mask_station = (df['Longitude'] < long_track + long_split_distance) & (df['Longitude'] >= long_track)
            
            latitude_mask_burn = (burn_data['LATITUDE'] < lat_track + lat_split_distance) & (burn_data['LATITUDE'] >= lat_track)
            longitude_mask_burn = (burn_data['LONGITUDE'] < long_track + long_split_distance) & (burn_data['LONGITUDE'] >= long_track)
            
            station_df = df[latitude_mask_station & longitude_mask_station]
            single_burn = burn_data[latitude_mask_burn & longitude_mask_burn]['FIRE_SIZE_HA'].sum(axis=0)
            total_burn.append(single_burn)
            
            print(f"total burn is {single_burn}")
            
            if single_burn >= 0.005: # in order to see number of regions with noticeable burn area
                num_above += 1
                
            if len(station_df) == 0: 
                zero_count += 1
            station_df.to_csv(f"data/weather-stations/kelowna/fine_area_{i*num_long_splits+j+1}_stations.csv")
            
            print(f"num of stations to aggregate is {len(station_df)}")
            
        long_track = longitude_min

    print(f"zero count is {zero_count}")
    print(f"num burn above 0.05 is {num_above}")

In [11]:
spatially_split(df, 12, 12)

min_long = -120.330555555556, min_lat = 49.5, max_long = -118.193333333333, max_lat = 51.0, long_split = 0.17810185185191685, lat_split = 0.125
total burn is 0.0
num of stations to aggregate is 2
total burn is 0.0
num of stations to aggregate is 1
total burn is 0.009
num of stations to aggregate is 2
total burn is 0.21800000000000003
num of stations to aggregate is 80
total burn is 0.15400000000000003
num of stations to aggregate is 6
total burn is 0.0
num of stations to aggregate is 3
total burn is 0.0
num of stations to aggregate is 0
total burn is 0.009
num of stations to aggregate is 3
total burn is 0.0
num of stations to aggregate is 1
total burn is 0.0
num of stations to aggregate is 1
total burn is 0.0
num of stations to aggregate is 0
total burn is 0.0
num of stations to aggregate is 1
total burn is 0.5
num of stations to aggregate is 7
total burn is 6.2
num of stations to aggregate is 5
total burn is 0.0
num of stations to aggregate is 2
total burn is 0.927
num of stations to 

In [224]:
print(total_burn)

[6.7, 0.009, 0.354, 1.9269999999999996, 0.009, 0.0, 0.6, 0.009, 0.0, 0.018, 2.127, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018, 0.509, 0.0, 4.2090000000000005, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.509, 0.009, 0.009, 0.0, 1.109, 0.026999999999999996, 0.009, 0.06, 3.1, 0.026999999999999996, 0.5, 4.009, 0.009, 0.018, 0.918, 0.927, 0.009, 0.079, 0.23600000000000002, 0.009, 0.545, 0.518, 0.44500000000000006, 0.009, 0.026999999999999996, 0.026999999999999996, 0.0, 0.0, 4.145, 69.027, 0.227, 0.127, 0.509, 0.8270000000000001, 0.026999999999999996, 0.018, 18.024]


In [231]:
# import os
# my_filter = ['ONE_DAY_PRECIPITATION', 'ONE_DAY_RAIN', 'MIN_TEMP', 'ONE_DAY_SNOW', 'time', 'MAX_TEMP']
filtered_df = df
for curr_year in range(2012, 2013):
    yearly_dataframes = []

    for index, row in filtered_df.iterrows():
        network_name = row['Network Name']
        native_id = row['Native ID']
        
        file_path = f'./data/weather/{curr_year}/{network_name}/{native_id}.csv'

        if os.path.exists(file_path):
            temp_df = pd.read_csv(file_path, skiprows=1)
            
            yearly_dataframes.append(temp_df)

    if yearly_dataframes:
        combined_df = pd.concat(yearly_dataframes, ignore_index=True)
        combined_df.columns = [i.lstrip() for i in combined_df.columns]
#         existing_columns = [c for c in my_filter if c in combined_df.columns]
#         combined_df = combined_df[existing_columns]
#       print(combined_df.columns)
        combined_csv_path = f'./training_data_20123/combined_data_{curr_year}.csv'
        combined_df.to_csv(combined_csv_path, index=False)
        print(combined_df)

       ONE_DAY_PRECIPITATION ONE_DAY_RAIN MIN_TEMP ONE_DAY_SNOW  \
0                        0.0          0.0     -4.0          0.0   
1                        0.0          0.0      0.0          0.0   
2                        4.2          4.2      3.0          0.0   
3                        0.0          0.0      3.5          0.0   
4                        0.0          0.0      0.0          0.0   
...                      ...          ...      ...          ...   
629306                   NaN          NaN      NaN          NaN   
629307                   NaN          NaN      NaN          NaN   
629308                   NaN          NaN      NaN          NaN   
629309                   NaN          NaN      NaN          NaN   
629310                   NaN          NaN      NaN          NaN   

                        time SNOW_ON_THE_GROUND MAX_TEMP AirTemp  \
0        2012-01-02 00:00:00                0.0      3.5     NaN   
1        2012-01-03 00:00:00                0.0      8.0   